# Sentiment Analysis in Twitter

#### Imports

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn import neighbors, model_selection, ensemble
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
import pickle

/Users/serg_pashkov/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/serg_pashkov/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


#### Get data

In [7]:
positive = pd.DataFrame()
negative = pd.DataFrame()

positive = pd.read_csv("positive.csv",sep=';',names= ['id','date','login','tweet','y','nReply','nRetvit','nFavorite'
                                                    ,'nUserMsgAmount','nFollowers','nFriends','nLists'])

negative = pd.read_csv("negative.csv",sep=';',names= ['id','date','login','tweet','y','nReply','nRetvit','nFavorite'
                                                      ,'nUserMsgAmount','nFollowers','nFriends','nLists'])
all_tweets = pd.DataFrame()
all_tweets = pd.concat([positive.head(3000),negative.head(3000)],ignore_index=True)
all_tweets = all_tweets.sample(frac=1)

#### Preprocessing

In [32]:
#Выравниваем регистр
def lower(str):
    return str.encode('utf-8').lower()
all_tweets['tweet'] = all_tweets.tweet.apply(lower)

In [10]:
#Разбиваем строку и удаляем обращения
def splitstring(str):
    words = []
    for i in re.split('[;,.,\n,\s,:,-,+,(,),=,/,«,»,@,\d,!,?,"]',str):
        if len(i) > 1:
            if re.match('@.*',i) == None:
                words.append(i)
    return words

In [11]:
all_tweets['Words'] = all_tweets.tweet.apply(splitstring)

In [12]:
#Разбиваем на выборки
x_train, x_test, y_train, y_test = model_selection.train_test_split(all_tweets.Words, all_tweets.y, test_size = 0.3)
print(x_train.size,y_train.size,x_test.size,y_test.size)

(4200, 4200, 1800, 1800)


In [13]:
#Создаем словарь
def WordsDic(dataset):
    WD = []
    for i in dataset.index:
        for j in xrange(len(dataset[i])):
            if dataset[i][j] in WD:
                None
            else:
                WD.append(dataset[i][j])
    return WD

words = WordsDic(x_train)

In [17]:
#Векторизируем текст
train_matrix = np.zeros((len(x_train),len(words)))

for i in xrange(train_matrix.shape[0]):
    for j in x_train[x_train.index[i]]:
        if j in words:
            train_matrix[i][words.index(j)]+=1

#### Model selection

In [26]:
#KNN
%time
param_grid = {'n_neighbors': np.arange(1,8), 'p': [1,2]}
estimator_kNN = neighbors.KNeighborsClassifier()
optimazer_kNN = GridSearchCV(estimator_kNN, param_grid, cv = 3)
optimazer_kNN.fit(train_matrix, y_train)

print optimazer_kNN.best_score_
print optimazer_kNN.best_params_

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 9.06 µs
0.525476190476
{'n_neighbors': 1, 'p': 2}


In [28]:
#RF
%time
param_grid = {'n_estimators': np.arange(20,101,10), 'min_samples_split': np.arange(4,11, 1)}
estimator_tree = ensemble.RandomForestClassifier()
optimazer_tree = GridSearchCV(estimator_tree, param_grid, cv = 3)
optimazer_tree.fit(train_matrix, y_train)

print optimazer_tree.best_score_
print optimazer_tree.best_params_

CPU times: user 5 µs, sys: 10 µs, total: 15 µs
Wall time: 31 µs
0.656428571429
{'min_samples_split': 8, 'n_estimators': 30}


#### Test

In [30]:
#Векторизируем текст
test_matrix = np.zeros((len(x_test),len(words)))
for i in xrange(test_matrix.shape[0]):
    for j in x_test[x_test.index[i]]:
        if j in words:
            test_matrix[i][words.index(j)]+=1

In [31]:
print u'Случайный лес:', accuracy_score(optimazer_tree.best_estimator_.predict(test_matrix), y_test)
print u'kNN:', accuracy_score(optimazer_kNN.best_estimator_.predict(test_matrix), y_test)

Случайный лес: 0.666666666667
kNN: 0.519444444444


In [34]:
#Сохраняем модель
joblib.dump(optimazer_tree, 'TweetForest.pkl')
joblib.dump(words, 'words.pkl')

['TweetForest.pkl']

In [ ]:
#Используем модель

clf = joblib.load('TweetForest.pkl')
words = joblib.load('words.pkl')

input_data = [ u'Круто, классно, обожаю, отлично, супер =)',
             u'Ненавижу эту сковородку, треснуло днище',
             u'На улице дождь, отстой',
             u'Завтра будет хорошая погода и мы пойдем на пикник',
             u'Ваша машинка ниочем']

inputDF = pd.DataFrame()
inputDF['tweet'] = input_data
inputDF['tweet'] = inputDF.tweet.apply(lambda x: x.lower())
inputDF['words'] = inputDF.tweet.apply(splitstring)
X = inputDF['words']


test_matrix = np.zeros((len(inputDF),len(words)))
for i in xrange(test_matrix.shape[0]):
    for j in X[X.index[i]]:
        if j in words:
            test_matrix[i][words.index(j)]+=1
            
predicted = clf.predict(test_matrix)
label_names = {1:u'Положительный', -1:u'Отрицательный'}

for tweet, rating in zip(input_data, predicted):
    print(u'\n{} ----> {}'.format(label_names.get(rating), tweet))